# 이미지 인식 - CNN

## MNIST 사례

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
# 데이터 불러오기
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')/255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')/255
Y_train = tf.keras.utils.to_categorical(Y_train)
Y_test = tf.keras.utils.to_categorical(Y_test)

In [4]:
# 컨볼루션 신경망 설정
# Conv2D(32 마스크의 갯수 kernel_size 마스크의 크기
#input_shape=(28,28,1) 행,열 컬러(3) 또는 흑백(1)
model = Sequential([
    Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3, 3), activation='selu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    Flatten(),
    Dense(500, activation='selu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 500)               4608500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
# 모델 최적화 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MEDEL_DIR)

In [7]:
modelpath = MODEL_DIR + "mnist-cnn-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [9]:
# 모델의 실행
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30,
                    batch_size=200, verbose=2, 
                    callbacks=[early_stopping_callback, checkpointer])

Train on 60000 samples, validate on 10000 samples
Epoch 1/30

Epoch 00001: val_loss improved from inf to 0.07363, saving model to ./model/mnist-cnn-01-0.0736.hdf5
60000/60000 - 85s - loss: 0.1665 - accuracy: 0.9495 - val_loss: 0.0736 - val_accuracy: 0.9756
Epoch 2/30

Epoch 00002: val_loss improved from 0.07363 to 0.05147, saving model to ./model/mnist-cnn-02-0.0515.hdf5
60000/60000 - 85s - loss: 0.0699 - accuracy: 0.9784 - val_loss: 0.0515 - val_accuracy: 0.9831
Epoch 3/30

Epoch 00003: val_loss did not improve from 0.05147
60000/60000 - 84s - loss: 0.0541 - accuracy: 0.9832 - val_loss: 0.0655 - val_accuracy: 0.9807
Epoch 4/30

Epoch 00004: val_loss improved from 0.05147 to 0.04409, saving model to ./model/mnist-cnn-04-0.0441.hdf5
60000/60000 - 85s - loss: 0.0455 - accuracy: 0.9856 - val_loss: 0.0441 - val_accuracy: 0.9868
Epoch 5/30

Epoch 00005: val_loss did not improve from 0.04409
60000/60000 - 84s - loss: 0.0382 - accuracy: 0.9876 - val_loss: 0.0499 - val_accuracy: 0.9855
Epoch 6

KeyError: 'val_loss'

In [ ]:
del model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('model/mnist-cnn-06-0.0384.hdf5')

In [ ]:
# 베스트 모델의 결과 출력 
model.evaluate(X_test, Y_test, verbose=0)[1]

In [ ]:
# 테스트셋의 오차
y_vloss = history.history['val_loss']

# 학습셋의 오차
y_loss = history.history['loss']

In [ ]:
# 그래프로 표현
x_len = np.arange(1, len(y_loss)+1)
plt.figure(figsize=(8,6))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
# plt.axis([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()